<a href="https://colab.research.google.com/github/da-head0/mypetfood/blob/main/Item_Based_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 참고
https://predictivehacks.com/item-based-collaborative-filtering-in-python/


In [9]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset/petfood.csv')
df.dropna(subset=['classification'], inplace=True) # 해당 열에 결측값이 있는 행 제거
df = df.reset_index(drop=True) # 인덱스 리셋
rating = pd.read_csv('/content/drive/MyDrive/dataset/catrating.csv')

In [2]:
df.isnull().sum()

category            0
brand               0
title               0
age                 0
classification      0
content             0
nutrient            0
info                9
gram                0
calory              7
ingredient          0
detail             21
from_company        0
성분등록번호            138
img                 0
dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/petfood.csv', index=False)

In [10]:
df = df[df.category == '고양이'] # 고양이만 남기기

In [13]:
df = df.reset_index(drop=True) # 인덱스 리셋

In [41]:
del df['detail']
del df['성분등록번호']

In [42]:
df.to_csv('catfood0607.csv', index=False)

In [14]:
df = df.reset_index()

In [ ]:
df = pd.read_csv('catfood0607.csv')

In [22]:
df.head(2)

,uuid,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img
0,0,고양이,뉴로 (NULO),뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩,"키튼,어덜트,시니어,전연령",건사료,"오리, 닭고기, 칠면조, 대구, 완두콩, 렌틸콩, 닭고기지방, 콩섬유질, 천연향, ...",조단백 40% 조지방 17% 칼슘 1.1% 인 0.9% 수분 10%,"그레인프리,글루텐프리,비유전자변형,유산균",5.4kg,"3,724 kcal/kg","오리, 닭고기, 칠면조","연령구분 키튼,어덜트,시니어,전연령 푸드타입 건사료 중량 5.4kg 주재료 오리, ...",미국/Nulo Inc,서울-23093호,https://cdn.purplesto.re/media/store/sale/main...
1,1,고양이,게더 (GATHER),프리에이커 치킨 어덜트,"어덜트,시니어",건사료,"유기농 닭고기, 유기농 건조 닭고기, 유기농 완두콩, 렌틸콩, 닭지방, 유기농 아마...",조단백 29% 조지방 15% 칼슘 1.4% 인 0.6% 수분 10%,"그레인프리,글루텐프리,비유전자변형",1.8kg,"3,664 kcal/kg","유기농 닭, 유기농 완두콩, 렌즈콩","연령구분 어덜트,시니어 푸드타입 건사료 중량 1.8kg 주재료 유기농 닭, 유기농 ...",캐나다/Petcurean Pet Nutrition Inc.,제 서울-19594호,https://cdn.purplesto.re/media/store/sale/main...


In [19]:
del df['index']

In [20]:
df = df.rename(columns={"level_0": "uuid"})

In [24]:
df['soup'] = df['brand'] +' ' + df['title']  +' ' + df['classification']  +' ' + df['content']  +' ' + df['info']
df.head(2)


,uuid,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img,soup
0,0,고양이,뉴로 (NULO),뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩,"키튼,어덜트,시니어,전연령",건사료,"오리, 닭고기, 칠면조, 대구, 완두콩, 렌틸콩, 닭고기지방, 콩섬유질, 천연향, ...",조단백 40% 조지방 17% 칼슘 1.1% 인 0.9% 수분 10%,"그레인프리,글루텐프리,비유전자변형,유산균",5.4kg,"3,724 kcal/kg","오리, 닭고기, 칠면조","연령구분 키튼,어덜트,시니어,전연령 푸드타입 건사료 중량 5.4kg 주재료 오리, ...",미국/Nulo Inc,서울-23093호,https://cdn.purplesto.re/media/store/sale/main...,"뉴로 (NULO) 뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩 건사료 오리, 닭고..."
1,1,고양이,게더 (GATHER),프리에이커 치킨 어덜트,"어덜트,시니어",건사료,"유기농 닭고기, 유기농 건조 닭고기, 유기농 완두콩, 렌틸콩, 닭지방, 유기농 아마...",조단백 29% 조지방 15% 칼슘 1.4% 인 0.6% 수분 10%,"그레인프리,글루텐프리,비유전자변형",1.8kg,"3,664 kcal/kg","유기농 닭, 유기농 완두콩, 렌즈콩","연령구분 어덜트,시니어 푸드타입 건사료 중량 1.8kg 주재료 유기농 닭, 유기농 ...",캐나다/Petcurean Pet Nutrition Inc.,제 서울-19594호,https://cdn.purplesto.re/media/store/sale/main...,"게더 (GATHER) 프리에이커 치킨 어덜트 건사료 유기농 닭고기, 유기농 건조 닭..."


In [ ]:
rating

In [25]:
import numpy as np
import sklearn
from sklearn.decomposition import TruncatedSVD

cat_rating = rating[['고양이_ID','rating', 'title']]
combined_foods_data = pd.merge(df,cat_rating, on='title', how='left') # 자동으로 inner가 되나.. 
combined_foods_data.head()

,uuid,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img,soup,고양이_ID,rating
0,0,고양이,뉴로 (NULO),뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩,"키튼,어덜트,시니어,전연령",건사료,"오리, 닭고기, 칠면조, 대구, 완두콩, 렌틸콩, 닭고기지방, 콩섬유질, 천연향, ...",조단백 40% 조지방 17% 칼슘 1.1% 인 0.9% 수분 10%,"그레인프리,글루텐프리,비유전자변형,유산균",5.4kg,"3,724 kcal/kg","오리, 닭고기, 칠면조","연령구분 키튼,어덜트,시니어,전연령 푸드타입 건사료 중량 5.4kg 주재료 오리, ...",미국/Nulo Inc,서울-23093호,https://cdn.purplesto.re/media/store/sale/main...,"뉴로 (NULO) 뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩 건사료 오리, 닭고...",NaN,NaN
1,1,고양이,게더 (GATHER),프리에이커 치킨 어덜트,"어덜트,시니어",건사료,"유기농 닭고기, 유기농 건조 닭고기, 유기농 완두콩, 렌틸콩, 닭지방, 유기농 아마...",조단백 29% 조지방 15% 칼슘 1.4% 인 0.6% 수분 10%,"그레인프리,글루텐프리,비유전자변형",1.8kg,"3,664 kcal/kg","유기농 닭, 유기농 완두콩, 렌즈콩","연령구분 어덜트,시니어 푸드타입 건사료 중량 1.8kg 주재료 유기농 닭, 유기농 ...",캐나다/Petcurean Pet Nutrition Inc.,제 서울-19594호,https://cdn.purplesto.re/media/store/sale/main...,"게더 (GATHER) 프리에이커 치킨 어덜트 건사료 유기농 닭고기, 유기농 건조 닭...",NaN,NaN
2,2,고양이,지위픽 (ZIWI),에어드라이 닭고기,"키튼,어덜트,시니어,전연령",저온건조 사료,"닭고기, 닭 내장(간, 심장) 및 뼈, 뉴질랜드 녹색홍합, 레시틴, 치커리 이눌린,...",조단백 44% 조지방 36% 칼슘 1.81% 인 1.38% 수분 8.0%,"그레인프리,글루텐프리",400g,"5,500 kcal/kg","닭고기, 닭 내장(간, 심장) 및 뼈, 뉴질랜드 녹색홍합","연령구분 키튼,어덜트,시니어,전연령 푸드타입 저온건조 사료 중량 400g 주재료 닭...",뉴질랜드/Ziwi Ltd,제 서울-15608호,https://cdn.purplesto.re/media/store/sale/main...,"지위픽 (ZIWI) 에어드라이 닭고기 저온건조 사료 닭고기, 닭 내장(간, 심장) ...",NaN,NaN
3,3,고양이,캣츠파인푸드 (CATZ FINEFOOD),주식파우치 no.15 닭고기와 꿩고기,"어덜트,시니어",습식사료,"닭, 물, 꿩, 쌀, 유채과, 미네랄, 민들레, 달맞이꽃 오일",조단백 8.6% 조지방 4% 칼슘 0.2% 인 0.15% 수분 88%,"검류미함유,비유전자변형",85g,965 kcal/kg,"닭, 물, 꿩","연령구분 어덜트,시니어 푸드타입 습식사료 중량 85g 주재료 닭, 물, 꿩 식단정보...",독일/Landguth Heimtiernahrung GmbH,GGHA50028호,https://cdn.purplesto.re/media/store/sale/main...,캣츠파인푸드 (CATZ FINEFOOD) 주식파우치 no.15 닭고기와 꿩고기 습식...,NaN,NaN
4,4,고양이,어플라우즈 (APPLAWS),파우치 닭고기와 아스파라거스,"키튼,어덜트,시니어,전연령",습식간식,"닭고기(가슴살), 닭육수, 아스파라거스, 쌀",조단백 14.0% 조지방 2.0% 조섬유 0.5% 조회분 1.0% 수분 82.0%,검류미함유,70g,532 kcal/kg (37.24 kcal/파우치),"닭고기(가슴살), 닭육수, 아스파라거스","연령구분 키튼,어덜트,시니어,전연령 푸드타입 습식간식 중량 70g 주재료 닭고기(가...",태국/MPM Products,제 서울-20530호,https://cdn.purplesto.re/media/store/sale/main...,어플라우즈 (APPLAWS) 파우치 닭고기와 아스파라거스 습식간식 닭고기(가슴살)...,NaN,NaN


In [31]:
df['soup'] = df['soup'].astype(str)

In [26]:
rating_crosstab = combined_foods_data.pivot_table(values='rating', 
                                                  index='고양이_ID', columns='soup', fill_value=0) # 없는 값은 0으로 채우기.

In [88]:
rating_crosstab

soup,"내추럴발란스 L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라 습식사료 닭고기, 닭고기 육즙, 호박, 카놀라 오일, 혼합 토코페롤 보존, 구아검, 인산칼슘, 천연 향, 소금, 황산칼슘, 염화칼륨, 칼슘, 인, 타우린, 비타민 C, E, D3, B12, 염화콜린, 카라기난, 로즈마리 추출물 알러지예방식","미유믹스 리얼 참치&새우 습식사료 참치, 생선육수, 바다생선 기호성좋은","사조 러브잇 연어&헤어볼 습식사료 다랑어, 연어, 증점안정제, 비타민E, 정제수 헤어볼 억제","아보덤 내추럴 그레인프리 정어리, 새우&게살 습식사료 정어리, 생선육수, 새우, 게, 타피오카, 해바라기유, 구아검, 제삼인산칼슘, 아보카도 오일, 타우린, 정제소금, 미네랄(산화 아연, 환원철, 아셀렌산나트륨, 황산망간, 요오드화칼륨) 염화칼륨, 염화콜린, 비타민 (비타민E, 나이아신, 치아민 질산염, 비타민 A분말) 등 그레인프리, 피부모질개선","아보덤 내추럴 참치+치킨 캔 습식사료 참치, 생선육수, 치킨, 해바라기 오일, 당근, PEA단백질, 제삼인산칼슘, 구아검, 아보카도오일, 타우린, 염화칼륨, 제1질산티아민, 나이아신, 판토텐산칼슘, 리보플라빈, 비타민E, 비타민A, 비타민B6, 비타민K3, 비타민D3, 비타민B12, 비오틴, 엽산 등 그레인프리","어게인스트 더 그레인 캡틴즈캐치 정어리&고등어 습식사료 정어리, 고등어, 물, 인산삼칼슘, 야자오일, 카라기닌, 잔탄검, 타우린, 비타민 및 미네랄 등 그레인프리, 글루텐프리, 비유전자변형","웰니스 코어 그레인프리 헬시 인덜전스 머셀 닭고기와 닭간 습식사료 닭고기육수, 정제수, 닭고기, 닭간, 감자전분, 달걀, 당근, 닭간, 천연향, 식염, 구아검, 제삼인산칼슘, 염화칼륨, 타우린, 황산마그네슘, 비타민, 크랜베리, 블루베리, 백색고구마, 잔탄검, 염화콜린, 아미노산킬레이트, 요오드화칼륨, 소듐셀레니트 그레인프리","웰니스 코어 시그니쳐 셀렉트 – 슈레드 닭고기와 닭간 습식사료 닭고기, 닭고기 육수, 충분한 물, 트로피카 전분, 감자 전분, 닭간, 닭걀, 천연향, 식염, 해바라기씨유, 3인산칼슘, 타우린, 황산 마그네슘, 구아검, 비타민[비타민E 첨가물, 염산치아민, 나이아신, 비타민A첨가물, 염산 피리독신, d-칼슘 판토텐산, 리보플라빈 첨가물, 엽산, 비타민D3 첨가물, 비오틴, 비타민B12첨가물], 염화칼륨, 염화콜린, 미네랄[킬레이트 아연, 킬레이트 철, 킬레이트 망간, 킬레이트 구리, 요오드화 칼륨, 아셀란산나트륨 그레인프리, 수제,","웰츠 캣 헤어볼 케어 건식사료 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리","카나간 프리런 치킨 포 캣 건식사료 닭고기, 건조육 닭고기, 고구마, 감자, 닭 지방, 건조란, 닭고기 그레이비, DHA 연어오일, 미네랄, 비타민(영양 첨가지), 알파파, 크랜베리, 만난올리고당, 사과, 당근, 시금치, 해초, 프락토올리고당, 카모마일, 페퍼민트, 마리골드, 아니스의 열매, 호로파 씨, 비타민 A, 비타민 D3, 비타민E, 타우린, 황산제일철, 황산아연, 황산망간, 황산동, 아셀렌산나트륨 그레인프리, 글루텐프리, 비유전자변형, 비유제품","캣만두 (CATMANDOO) 라이프 에센셜 스프링클스 치킨 토퍼(부스터) 닭고기 100% 그레인프리,글루텐프리","퓨어비타 (PUREVITA) 그레인프리 치킨 건사료 닭고기, 닭고기분, 완두콩, 완두콩 가루, 닭 지방, 계란분말, 천연향, 효모제(맥주 효모), 식염류, 아미노산제(DL-메치오닌), 근괴류(당근), 해바라기유, 미량광물질류합제(철, 아연, 구리, 망간, 코발트, 셀레늄), 핵과류(체리, 살구), 장과류(크랜베리, 블루베리), 근채류(치커리), 비타민합제(A, D3, E, 니아신, 판토텐산칼슘, 티아민, 리보플라빈, 엽산, 비오틴, B12, C, 콜린, 이노시톨), 생균제(락토 바실러스 애시도필러스, 바실러스서브틸리스), 유익곰팡이균(아스퍼질러스 오리제) 그레인프리,글루텐프리","피니키 참치무스 캔 습식사료 참치, 정제수, 타피오카전분, 참치 추출물, 해조류(진두발추출물), 구아검, 염화칼륨, 비타민E 수분함량높음","힐스 사이언스다이어트 어덜트 유리너리&헤어볼컨트롤 건식사료 닭, 통밀, 옥수수글루텐박, 돼지지방, 분말셀루로오스, 밀글루텐, 닭간향료, 건조사탕무박, 대두유, 젖산, 황산칼슘, 어유, 염화콜린, 염화칼륨, L-라이신, 요오드첨가식염, 인산이칼슘, 타우린, 비타민합제(비타민E보충제, 비타민A보충제), 구연산칼륨, 미량광물질합제(황산제일철, 산화아연), L-카르니틴, 혼합토코페롤, 천연향료, 녹색완두콩, 사과, 크랜베리, 당근, 브로콜리, 베타카로틴 헤어볼 억제, 신장 결석 예방,"
고양이_ID,,,,,,,,,,,,,,
1.0,2,5,5,1,0,5,0,0,5,0,0,0,1,4
2.0,0,0,0,0,1,0,5,5,0,5,0,1,0,0
3.0,0,0,0,0,3,0,0,0,0,0,5,0,0,0


In [109]:
rating_crosstab['favorite'] = ""
rating_crosstab['soso'] = ""
rating_crosstab['no'] = ""

for i in range(len(rating_crosstab)):
  for j in range(len(rating_crosstab.columns)):
    try:
      if int(rating_crosstab.iloc[i,j]) >= 4 : # 2,5,5,1,0,5,0,...
        rating_crosstab.iloc[i,-3] += rating_crosstab.columns[j] + ' '
      elif 3<= int(rating_crosstab.iloc[i,j]) <4 :
        rating_crosstab.iloc[i,-2] += rating_crosstab.columns[j] + ' '
      elif 1<= int(rating_crosstab.iloc[i,j]) < 3:
        rating_crosstab.iloc[i,-1] += rating_crosstab.columns[j] + ' '
      else:
        pass
    except: pass

rating_crosstab.T

고양이_ID,1.0,2.0,3.0
soup,,,
"내추럴발란스 L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라 습식사료 닭고기, 닭고기 육즙, 호박, 카놀라 오일, 혼합 토코페롤 보존, 구아검, 인산칼슘, 천연 향, 소금, 황산칼슘, 염화칼륨, 칼슘, 인, 타우린, 비타민 C, E, D3, B12, 염화콜린, 카라기난, 로즈마리 추출물 알러지예방식",2,0,0
"미유믹스 리얼 참치&새우 습식사료 참치, 생선육수, 바다생선 기호성좋은",5,0,0
"사조 러브잇 연어&헤어볼 습식사료 다랑어, 연어, 증점안정제, 비타민E, 정제수 헤어볼 억제",5,0,0
"아보덤 내추럴 그레인프리 정어리, 새우&게살 습식사료 정어리, 생선육수, 새우, 게, 타피오카, 해바라기유, 구아검, 제삼인산칼슘, 아보카도 오일, 타우린, 정제소금, 미네랄(산화 아연, 환원철, 아셀렌산나트륨, 황산망간, 요오드화칼륨) 염화칼륨, 염화콜린, 비타민 (비타민E, 나이아신, 치아민 질산염, 비타민 A분말) 등 그레인프리, 피부모질개선",1,0,0
"아보덤 내추럴 참치+치킨 캔 습식사료 참치, 생선육수, 치킨, 해바라기 오일, 당근, PEA단백질, 제삼인산칼슘, 구아검, 아보카도오일, 타우린, 염화칼륨, 제1질산티아민, 나이아신, 판토텐산칼슘, 리보플라빈, 비타민E, 비타민A, 비타민B6, 비타민K3, 비타민D3, 비타민B12, 비오틴, 엽산 등 그레인프리",0,1,3
"어게인스트 더 그레인 캡틴즈캐치 정어리&고등어 습식사료 정어리, 고등어, 물, 인산삼칼슘, 야자오일, 카라기닌, 잔탄검, 타우린, 비타민 및 미네랄 등 그레인프리, 글루텐프리, 비유전자변형",5,0,0
"웰니스 코어 그레인프리 헬시 인덜전스 머셀 닭고기와 닭간 습식사료 닭고기육수, 정제수, 닭고기, 닭간, 감자전분, 달걀, 당근, 닭간, 천연향, 식염, 구아검, 제삼인산칼슘, 염화칼륨, 타우린, 황산마그네슘, 비타민, 크랜베리, 블루베리, 백색고구마, 잔탄검, 염화콜린, 아미노산킬레이트, 요오드화칼륨, 소듐셀레니트 그레인프리",0,5,0
"웰니스 코어 시그니쳐 셀렉트 – 슈레드 닭고기와 닭간 습식사료 닭고기, 닭고기 육수, 충분한 물, 트로피카 전분, 감자 전분, 닭간, 닭걀, 천연향, 식염, 해바라기씨유, 3인산칼슘, 타우린, 황산 마그네슘, 구아검, 비타민[비타민E 첨가물, 염산치아민, 나이아신, 비타민A첨가물, 염산 피리독신, d-칼슘 판토텐산, 리보플라빈 첨가물, 엽산, 비타민D3 첨가물, 비오틴, 비타민B12첨가물], 염화칼륨, 염화콜린, 미네랄[킬레이트 아연, 킬레이트 철, 킬레이트 망간, 킬레이트 구리, 요오드화 칼륨, 아셀란산나트륨 그레인프리, 수제,",0,5,0
"웰츠 캣 헤어볼 케어 건식사료 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리",5,0,0


In [115]:
# 일단 미모의 입맛을 나눠보자
fav = rating_crosstab.loc[1.0,'favorite']
ss = rating_crosstab.loc[1.0,'soso']
nn = rating_crosstab.loc[1.0,'no']

In [ ]:
# - 고양이마다 - 고양이 ID로 거르기
# - 좋아하는 식품, 소소, No를 합친다? no는 패널티를 주고. (Vectorizer에 패널티를 줄 수 있나?) 
# - 빈도수를 세되 5점은 * 5 3점은 *3 1점은 *1 하고 평균내기
# - 기존 전체 식품들과 코사인 유사도 비교

In [127]:
from math import log # IDF 계산을 위해
docs = [fav, ss, nn]
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [128]:
N = len(docs)

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for f in fav:
    df += t in f
  return log(N)/(df+1)

def tfidf(t, d):
  return tf(t,d) * idf(t)

In [130]:
# tf 구하기 - DTM 데이터프레임에 저장하여 출력

result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t,d)*5)

tf_ = pd.DataFrame(result, columns = vocab, index=['fav', 'soso', 'no'])
tf_

,&,"(비타민E,",A분말),"B1,","B12,","B2,","B6,","C,","D3,","E,","L-라이신,","L-카르니틴,",L.I.D.,건식사료,건조,"건조사탕무박,","건조유카,","건조정어리,","게,",결석,"고구마,","고등어,","구리,","구아검,","구연산칼륨,",그레인,그레인프리,"그레인프리,","글루코사민,","글루텐프리,",기호성좋은,"나이아신,",내추럴,내추럴발란스,"녹색완두콩,","다랑어,","닭,","닭간,","닭간향료,",닭고기,...,"추출물,",치아민,카놀라,"카라기난,","카라기닌,","카로틴,","칼슘,",캔,캡틴즈캐치,캣,케어,"코발트,","코코넛오일,","콘드로이틴,","크랜베리,","타우린,","타임,","타피오카,","타피오카전분,",토코페롤,"통밀,","판토텐산,","페뉴그릭,",포뮬라,"프로바이오틱스,","프리바이오틱스,",피니키,피부모질개선,"해바라기유,","해조류(진두발추출물),","향,",헤어볼,호박,"호박,",혼합,"혼합토코페롤,","환원철,","황산망간,","황산칼슘,",힐스
fav,20,0,0,10,10,10,10,10,10,15,5,5,0,10,35,5,10,10,0,5,10,5,10,0,5,10,5,5,10,5,5,10,0,0,5,5,5,10,5,20,...,0,0,0,0,5,10,15,0,5,5,5,10,10,10,5,20,10,0,0,5,5,10,10,0,10,10,0,0,0,0,0,25,0,0,5,5,0,0,5,5
soso,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
no,10,5,5,0,5,0,0,5,5,10,0,0,5,0,0,0,0,0,5,0,0,0,0,15,0,10,10,5,0,0,0,5,10,5,0,0,0,0,0,15,...,5,5,5,5,0,0,20,5,0,5,0,0,0,0,0,10,0,5,5,5,0,0,0,5,0,0,5,5,5,5,5,0,10,5,5,0,5,5,5,0


In [132]:
tfsum = tf_.T
tfsum.head(3)

,fav,soso,no
&,20,0,10
"(비타민E,",0,0,5
A분말),0,0,5


In [135]:
tfsum['sum'] = ((tfsum['fav'] * 3) - (tfsum['soso'] * 2) - (tfsum['no'] * 1)) / 6
tfsum

,fav,soso,no,sum
&,20,0,10,8.333333
"(비타민E,",0,0,5,-0.833333
A분말),0,0,5,-0.833333
"B1,",10,0,0,5.000000
"B12,",10,0,5,4.166667
...,...,...,...,...
"혼합토코페롤,",5,0,0,2.500000
"환원철,",0,0,5,-0.833333
"황산망간,",0,0,5,-0.833333
"황산칼슘,",5,0,5,1.666667


- 는 별로 안 좋아한다는 거다. 이게 들어가면 안됨. 

In [136]:
avgdtm = tfsum['sum']
avgdtm

&          8.333333
(비타민E,    -0.833333
A분말)      -0.833333
B1,        5.000000
B12,       4.166667
             ...   
혼합토코페롤,    2.500000
환원철,      -0.833333
황산망간,     -0.833333
황산칼슘,      1.666667
힐스         2.500000
Name: sum, Length: 189, dtype: float64

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix = count.fit_transform(df['soup'])

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [55]:
df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])

In [84]:
idx = indices['프리에이커 치킨 어덜트']

# Get the pairwsie similarity scores of all movies with that movie
sim_scores = list(enumerate(cosine_sim[idx]))

# Sort the movies based on the similarity scores
sim_scores.sort()#key=lambda x:-x[1])#, key=lambda x: x[1], reverse=True)

In [85]:
# Get the scores of the 3 most similar movies
sim_scores = sim_scores[1:10] # 10개도 가능

# Get the movie indices
movie_indices = [i[0] for i in sim_scores]

df['title'].iloc[movie_indices][1:]

2               에어드라이 닭고기
3    주식파우치 no.15 닭고기와 꿩고기
Name: title, dtype: object

In [29]:
rating_crosstab

soup,"내추럴발란스 L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라 습식사료 닭고기, 닭고기 육즙, 호박, 카놀라 오일, 혼합 토코페롤 보존, 구아검, 인산칼슘, 천연 향, 소금, 황산칼슘, 염화칼륨, 칼슘, 인, 타우린, 비타민 C, E, D3, B12, 염화콜린, 카라기난, 로즈마리 추출물 알러지예방식","미유믹스 리얼 참치&새우 습식사료 참치, 생선육수, 바다생선 기호성좋은","사조 러브잇 연어&헤어볼 습식사료 다랑어, 연어, 증점안정제, 비타민E, 정제수 헤어볼 억제","아보덤 내추럴 그레인프리 정어리, 새우&게살 습식사료 정어리, 생선육수, 새우, 게, 타피오카, 해바라기유, 구아검, 제삼인산칼슘, 아보카도 오일, 타우린, 정제소금, 미네랄(산화 아연, 환원철, 아셀렌산나트륨, 황산망간, 요오드화칼륨) 염화칼륨, 염화콜린, 비타민 (비타민E, 나이아신, 치아민 질산염, 비타민 A분말) 등 그레인프리, 피부모질개선","아보덤 내추럴 참치+치킨 캔 습식사료 참치, 생선육수, 치킨, 해바라기 오일, 당근, PEA단백질, 제삼인산칼슘, 구아검, 아보카도오일, 타우린, 염화칼륨, 제1질산티아민, 나이아신, 판토텐산칼슘, 리보플라빈, 비타민E, 비타민A, 비타민B6, 비타민K3, 비타민D3, 비타민B12, 비오틴, 엽산 등 그레인프리","어게인스트 더 그레인 캡틴즈캐치 정어리&고등어 습식사료 정어리, 고등어, 물, 인산삼칼슘, 야자오일, 카라기닌, 잔탄검, 타우린, 비타민 및 미네랄 등 그레인프리, 글루텐프리, 비유전자변형","웰니스 코어 그레인프리 헬시 인덜전스 머셀 닭고기와 닭간 습식사료 닭고기육수, 정제수, 닭고기, 닭간, 감자전분, 달걀, 당근, 닭간, 천연향, 식염, 구아검, 제삼인산칼슘, 염화칼륨, 타우린, 황산마그네슘, 비타민, 크랜베리, 블루베리, 백색고구마, 잔탄검, 염화콜린, 아미노산킬레이트, 요오드화칼륨, 소듐셀레니트 그레인프리","웰니스 코어 시그니쳐 셀렉트 – 슈레드 닭고기와 닭간 습식사료 닭고기, 닭고기 육수, 충분한 물, 트로피카 전분, 감자 전분, 닭간, 닭걀, 천연향, 식염, 해바라기씨유, 3인산칼슘, 타우린, 황산 마그네슘, 구아검, 비타민[비타민E 첨가물, 염산치아민, 나이아신, 비타민A첨가물, 염산 피리독신, d-칼슘 판토텐산, 리보플라빈 첨가물, 엽산, 비타민D3 첨가물, 비오틴, 비타민B12첨가물], 염화칼륨, 염화콜린, 미네랄[킬레이트 아연, 킬레이트 철, 킬레이트 망간, 킬레이트 구리, 요오드화 칼륨, 아셀란산나트륨 그레인프리, 수제,","웰츠 캣 헤어볼 케어 건식사료 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리","카나간 프리런 치킨 포 캣 건식사료 닭고기, 건조육 닭고기, 고구마, 감자, 닭 지방, 건조란, 닭고기 그레이비, DHA 연어오일, 미네랄, 비타민(영양 첨가지), 알파파, 크랜베리, 만난올리고당, 사과, 당근, 시금치, 해초, 프락토올리고당, 카모마일, 페퍼민트, 마리골드, 아니스의 열매, 호로파 씨, 비타민 A, 비타민 D3, 비타민E, 타우린, 황산제일철, 황산아연, 황산망간, 황산동, 아셀렌산나트륨 그레인프리, 글루텐프리, 비유전자변형, 비유제품","캣만두 (CATMANDOO) 라이프 에센셜 스프링클스 치킨 토퍼(부스터) 닭고기 100% 그레인프리,글루텐프리","퓨어비타 (PUREVITA) 그레인프리 치킨 건사료 닭고기, 닭고기분, 완두콩, 완두콩 가루, 닭 지방, 계란분말, 천연향, 효모제(맥주 효모), 식염류, 아미노산제(DL-메치오닌), 근괴류(당근), 해바라기유, 미량광물질류합제(철, 아연, 구리, 망간, 코발트, 셀레늄), 핵과류(체리, 살구), 장과류(크랜베리, 블루베리), 근채류(치커리), 비타민합제(A, D3, E, 니아신, 판토텐산칼슘, 티아민, 리보플라빈, 엽산, 비오틴, B12, C, 콜린, 이노시톨), 생균제(락토 바실러스 애시도필러스, 바실러스서브틸리스), 유익곰팡이균(아스퍼질러스 오리제) 그레인프리,글루텐프리","피니키 참치무스 캔 습식사료 참치, 정제수, 타피오카전분, 참치 추출물, 해조류(진두발추출물), 구아검, 염화칼륨, 비타민E 수분함량높음","힐스 사이언스다이어트 어덜트 유리너리&헤어볼컨트롤 건식사료 닭, 통밀, 옥수수글루텐박, 돼지지방, 분말셀루로오스, 밀글루텐, 닭간향료, 건조사탕무박, 대두유, 젖산, 황산칼슘, 어유, 염화콜린, 염화칼륨, L-라이신, 요오드첨가식염, 인산이칼슘, 타우린, 비타민합제(비타민E보충제, 비타민A보충제), 구연산칼륨, 미량광물질합제(황산제일철, 산화아연), L-카르니틴, 혼합토코페롤, 천연향료, 녹색완두콩, 사과, 크랜베리, 당근, 브로콜리, 베타카로틴 헤어볼 억제, 신장 결석 예방,"
고양이_ID,,,,,,,,,,,,,,
1.0,2,5,5,1,0,5,0,0,5,0,0,0,1,4
2.0,0,0,0,0,1,0,5,5,0,5,0,1,0,0
3.0,0,0,0,0,3,0,0,0,0,0,5,0,0,0


In [27]:
X = rating_crosstab.T

In [28]:
X

고양이_ID,1.0,2.0,3.0
soup,,,
"내추럴발란스 L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라 습식사료 닭고기, 닭고기 육즙, 호박, 카놀라 오일, 혼합 토코페롤 보존, 구아검, 인산칼슘, 천연 향, 소금, 황산칼슘, 염화칼륨, 칼슘, 인, 타우린, 비타민 C, E, D3, B12, 염화콜린, 카라기난, 로즈마리 추출물 알러지예방식",2,0,0
"미유믹스 리얼 참치&새우 습식사료 참치, 생선육수, 바다생선 기호성좋은",5,0,0
"사조 러브잇 연어&헤어볼 습식사료 다랑어, 연어, 증점안정제, 비타민E, 정제수 헤어볼 억제",5,0,0
"아보덤 내추럴 그레인프리 정어리, 새우&게살 습식사료 정어리, 생선육수, 새우, 게, 타피오카, 해바라기유, 구아검, 제삼인산칼슘, 아보카도 오일, 타우린, 정제소금, 미네랄(산화 아연, 환원철, 아셀렌산나트륨, 황산망간, 요오드화칼륨) 염화칼륨, 염화콜린, 비타민 (비타민E, 나이아신, 치아민 질산염, 비타민 A분말) 등 그레인프리, 피부모질개선",1,0,0
"아보덤 내추럴 참치+치킨 캔 습식사료 참치, 생선육수, 치킨, 해바라기 오일, 당근, PEA단백질, 제삼인산칼슘, 구아검, 아보카도오일, 타우린, 염화칼륨, 제1질산티아민, 나이아신, 판토텐산칼슘, 리보플라빈, 비타민E, 비타민A, 비타민B6, 비타민K3, 비타민D3, 비타민B12, 비오틴, 엽산 등 그레인프리",0,1,3
"어게인스트 더 그레인 캡틴즈캐치 정어리&고등어 습식사료 정어리, 고등어, 물, 인산삼칼슘, 야자오일, 카라기닌, 잔탄검, 타우린, 비타민 및 미네랄 등 그레인프리, 글루텐프리, 비유전자변형",5,0,0
"웰니스 코어 그레인프리 헬시 인덜전스 머셀 닭고기와 닭간 습식사료 닭고기육수, 정제수, 닭고기, 닭간, 감자전분, 달걀, 당근, 닭간, 천연향, 식염, 구아검, 제삼인산칼슘, 염화칼륨, 타우린, 황산마그네슘, 비타민, 크랜베리, 블루베리, 백색고구마, 잔탄검, 염화콜린, 아미노산킬레이트, 요오드화칼륨, 소듐셀레니트 그레인프리",0,5,0
"웰니스 코어 시그니쳐 셀렉트 – 슈레드 닭고기와 닭간 습식사료 닭고기, 닭고기 육수, 충분한 물, 트로피카 전분, 감자 전분, 닭간, 닭걀, 천연향, 식염, 해바라기씨유, 3인산칼슘, 타우린, 황산 마그네슘, 구아검, 비타민[비타민E 첨가물, 염산치아민, 나이아신, 비타민A첨가물, 염산 피리독신, d-칼슘 판토텐산, 리보플라빈 첨가물, 엽산, 비타민D3 첨가물, 비오틴, 비타민B12첨가물], 염화칼륨, 염화콜린, 미네랄[킬레이트 아연, 킬레이트 철, 킬레이트 망간, 킬레이트 구리, 요오드화 칼륨, 아셀란산나트륨 그레인프리, 수제,",0,5,0
"웰츠 캣 헤어볼 케어 건식사료 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리 닭고기, 닭고기분, 병아리콩, 연어, 오리, 건조정어리, 고구마, 완두콩, 천연 셀룰로스, 알파파, 닭간, 천연 닭기름, 렌틸콩, 바나나, 아마씨, 연어오일, 코코넛오일, 비타민A, D3, E, B1, B2, B6, B12, 바이오틴, 엽산, 판토텐산, 나이아신, 미네랄(철, 아연, 망간, 구리, 요오드, 코발트, 셀레늄), 타우린, 베타 카로틴, 비타민C, 심황, 타임, 페뉴그릭, 이스트, 프로바이오틱스, 프리바이오틱스, 건조유카, 글루코사민, 콘드로이틴, 마키베리, 아로니아베리, 아사이베리, 건조 로즈마리",5,0,0


- 역시 한번이라도 평가한 데이터가 추천된다.. 흠...

In [ ]:
# 지금은 평가 고양이 수가 작아서 안 해도 될 것 같음. 
# n_components는 평가 고양이 수보다 작아야 함.
SVD = TruncatedSVD(n_components=2, random_state=5)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(19, 2)

In [ ]:
### correlation matrix
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(19, 19)

In [ ]:
col_idx = rating_crosstab.columns.get_loc("참치무스 캔")
corr_specific = corr_mat[col_idx]
pd.DataFrame({'corr_specific':corr_specific, 'foods': rating_crosstab.columns})\
.sort_values('corr_specific', ascending=False)\
.head(10)

,corr_specific,foods
9,1.0,리얼 참치&새우
2,1.0,"내추럴 그레인프리 정어리, 새우&게살"
16,1.0,퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비
13,1.0,캣 헤어볼 케어
12,1.0,캡틴즈캐치 정어리&고등어
6,1.0,러브잇 다랑어흰살
7,1.0,러브잇 맛살
8,1.0,러브잇 연어&헤어볼
1,1.0,L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라
11,1.0,참치무스 캔


In [ ]:
col_idx = rating_crosstab.columns.get_loc("캡틴즈캐치 정어리&고등어")
corr_specific = corr_mat[col_idx]
pd.DataFrame({'corr_specific':corr_specific, 'foods': rating_crosstab.columns})\
.sort_values('corr_specific', ascending=False)\
.head(5)

,corr_specific,foods
11,1.0,참치무스 캔
2,1.0,"내추럴 그레인프리 정어리, 새우&게살"
16,1.0,퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비
1,1.0,L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라
10,1.0,사이언스다이어트 어덜트 유리너리&헤어볼컨트롤


- 어제보다 나은 건 그래도 자기 자신이 아닌, 평가한 게 아닌 다른 아이템이 나오기라도 한다는 것이다...
- 하지만 평가에만 의존해서 하는 것은.. 흠...

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 2.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617620 sha256=96837f411f2d22f4ee6a7c67c20ccb8b6b17a001351373e390ac73d8c0e955a5
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

# 여기는 rating이 df

reader = Reader(rating_scale=(1,5)) # to specify rating scale
data = Dataset.load_from_df(rating[['고양이_ID','title','rating']], reader)
#df = Dataset.load_from_file('/content/drive/MyDrive/dataset/petfood.csv', reader=)

In [ ]:
combined_foods_data

In [ ]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

sim_options = {"name" : ["msd", "cosine"],
               "min_support" : [3,4,5],
               "user_based" : [False, True]} #  # Compute  similarities between items
param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

1.7941707695742355
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
prediction = algo.predict(1, '피스트 캔 양고기와 연어')
prediction.est

Computing the cosine similarity matrix...
Done computing similarity matrix.


3.0714285714285716

In [ ]:
prediction = gs.predict(1, '피스트 캔 양고기와 연어')
prediction.est

ValueError: ignored

- 한 숫자로 찍는데??

In [ ]:
rating

,고양이_ID,고양이_이름,고양이_나이,title,rating
0,1,미모,1,사이언스다이어트 어덜트 유리너리&헤어볼컨트롤,4
1,1,미모,1,캣 헤어볼 케어,5
2,1,미모,1,피스트 캔 양고기와 연어,5
3,1,미모,1,주식캔 퓨레 no.109 돼지고기,5
4,1,미모,1,주식캔 필렛 no.403 닭고기,5
5,1,미모,1,"주식캔 필렛 no.409 칠면조, 닭고기와 토끼",5
6,1,미모,1,HFC 내츄럴플러스 파우치 대서양참치,5
7,1,미모,1,주식 파우치 참치&연어,5
8,1,미모,1,캡틴즈캐치 정어리&고등어,5
9,1,미모,1,러브잇 다랑어흰살,5
